In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
import tqdm

In [9]:
# the datasets are too large, do not push to github!
# download the file on the desktop
mypath = "C:/data"
files = [mypath+'/'+str(f) for f in listdir(mypath) if isfile(join(mypath, f))]


Create data dictionary and move these 5 files into it.

Output of listdir(mypath) should be:

data/members_v3.csv.7z 

data/sample_submission_zero.csv.7z

data/train.csv.7z

data/transactions.csv.7z

data/user_logs.csv.7z

In [10]:
files

['C:/data/members_v3.csv.7z',
 'C:/data/sample_submission_v2.csv.7z',
 'C:/data/train.csv.7z',
 'C:/data/transactions.csv.7z',
 'C:/data/user_logs.csv.7z']

In [16]:
if "unpacked_data" not in listdir('./'):
    os.makedirs("./unpacked_data")
else:
    print("Dictionary is already created")

Dictionary is already created


In [ ]:
# unpack data
import py7zr


extract_path = "unpacked_data"
for file in tqdm.tqdm(files):
    with py7zr.SevenZipFile(file, mode='r') as z:
        data = z.extractall(extract_path)

In [61]:
'''
Functions that use to change the datatype according to columns' largest values for memory saving。

For integer:
For example, the maximum number that int8 can store is 127, and the minimum is - 128;
if the largest values for such column is smaller than the value，
we change the data type to int8
Perform this for all columns include int.

For float:
We change all features to float32
Source: https://www.kaggle.com/jeru666/did-you-think-of-these-features/notebook
'''

def change_datatype(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
            
def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)

# Merging data

In [60]:
train = pd.read_csv(r"C:\Users\Shileng Zhang\Documents\GitHub\STA142A-Project\.ipynb_checkpoints\unpacked_data\train.csv")
user = pd.read_csv(r"C:\Users\Shileng Zhang\Documents\GitHub\STA142A-Project\.ipynb_checkpoints\unpacked_data\user_logs.csv", nrows = 1000000)
member = pd.read_csv(r"C:\Users\Shileng Zhang\Documents\GitHub\STA142A-Project\.ipynb_checkpoints\unpacked_data\members_v3.csv", nrows = 1000000)
transactions = pd.read_csv(r"C:\Users\Shileng Zhang\Documents\GitHub\STA142A-Project\.ipynb_checkpoints\unpacked_data\transactions.csv", nrows = 1000000)

In [215]:
#Example
mem = train.memory_usage(index=True).sum()
print("current usage:", mem/ 1024**2," MB")

current usage: 15.151046752929688  MB


In [217]:
#change data type
change_datatype(train)
mem = train.memory_usage(index=True).sum()
print("usage after type changes:", mem/ 1024**2," MB")

usage after type changes: 8.522517204284668  MB


In [230]:
#merge data
train1 = pd.merge(train, user, on = 'msno') 
train2 = pd.merge(train, member, on = 'msno') 
train3 = pd.merge(train, transactions, on = 'msno') 

In [231]:
change_datatype(train1)
change_datatype_float(train2)
change_datatype(train3)

# Random Forest

In [183]:
#import packages
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [241]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [267]:
# Userlog
X = train1.drop('is_churn',axis = 1)
X = X.drop('msno', axis = 1)
Y = train1['is_churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
X_train.shape, Y_train.shape

((437356, 8), (437356,))

In [268]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)

In [269]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.943352237261189


In [259]:
# for member
train2 = train2.iloc[0:,1:]
clean_dataset(train2)
train2['gender'].replace({'female':'1', 'male':'0'}, inplace = True)
X = train2.drop('is_churn',axis = 1)
Y = train2['is_churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
X_train.shape, Y_train.shape

((53976, 5), (53976,))

In [260]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)

In [261]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.9018285566074439


In [232]:
# for transactions
X = train3.drop('is_churn',axis = 1)
X = X3.drop('msno', axis = 1)
Y = train3['is_churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
X_train.shape, Y_train.shape

((515993, 8), (515993,))

In [233]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)

In [234]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.9545448379088455
